# 04. NumPy Project: PageRank

August 26, 2018

We've now learned enough to be dangerous! 

Let's use this knowledge to implement two famous algorithms:

* **Ranking Web Pages with PageRank**
* Predicting Emojis in Tweets with Linear Regression

In [ ]:
import numpy as np
print(f'numpy version: {np.__version__}')

## PageRank

PageRank is one of the most lucrative algorithms in history, used by Google to revolutionize how web search results are ranked. Let's go over the math behind PageRank, and then implement it in NumPy.

PageRank paper:  http://infolab.stanford.edu/pub/papers/google.pdf 

Wikipedia article:  https://en.wikipedia.org/wiki/PageRank

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/PageRank-hi-res.png/440px-PageRank-hi-res.png" width="600"/>

## PageRank Basics
From the original paper:

>PageRank can be thought of as a model of user behavior. We assume there is a \"random surfer\" who is
given a web page at random and keeps clicking on links, never hitting \"back\" but eventually gets bored
and starts on another random page. The probability that the random surfer visits a page is its PageRank.
And, the d damping factor is the probability at each page the \"random surfer\" will get bored and request
another random page.

There is one mistake in this description: the damping factor $d$ is instead the probability that the surfer keeps clicking links. The description above would be $(1-d)$.

The core data structure for PageRank is a hyperlink graph, where web pages are represented as nodes and a hyperlink from page j to page i forms a directed edge in the graph from node j to node i. The picture below shows a toy "web" with  11 web pages and the link structure between them. We'll work our way towards implementing PageRank on this graph.

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/8/8b/PageRanks-Example.jpg/800px-PageRanks-Example.jpg" width="600"/>

## A Random Walk Down Wall Street

First we're going to implement PageRank the "naive" way by simulating a random walk. To keep things simple, let's consider a tiny web with three pages: A, B, and C.

<img src="graph3.png" width="200"/>

* A links to B
* B links to C
* C links A and B

First let's implement a function that, given a page and a graph, returns a random "next page". If a page has no outgoing links, we will select the next page completely at random.

Some methods to consider:

* np.random.randint(), https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.randint.html
* np.random.choice(), https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html

In [ ]:
# Sample Implementation

def random_next_page(current_page, link_graph):
    """
    If the current_page has no outgoing links, a random page is returned.

    :param current_page: a string with the current page name
    :param link_graph: a dict mapping page names to "next page" names, e.g. {"page1": ["page2, page5"]}
    :return: a string with the next page
    """
    page_choices = link_graph[current_page]
    if len(page_choices) == 0:
        page_choices = list(link_graph.keys())
    return np.random.choice(page_choices)

In [ ]:
def random_next_page(current_page, link_graph):
    """
    If the current_page has no outgoing links, a random page is returned.

    :param current_page: a string with the current page name
    :param link_graph: a dict mapping page names to "next page" names, e.g. {"page1": ["page2, page5"]}
    :return: a string with the next page
    """
    raise Exception('Implement me!')

Now let's implement a method that, given a damping factor, determines if we should keep surfing.

In [ ]:
# Sample Implementation

def keep_surfing(damping_factor):
    """
    :param damping_factor: the probability that the surfer keeps surfing
    :return: True if the surfer should keep surfing, False otherwise
    """
    return np.random.rand() < damping_factor

In [ ]:
def keep_surfing(damping_factor):
    """
    :param damping_factor: the probability that the surfer keeps surfing
    :return: True if the surfer should keep surfing, False otherwise
    """
    raise Exception('Implement me!')

Finally, using the two functions you just created, implement a function that performs a random walk given a starting page, and keeps track of page visits using a Counter object.

In [ ]:
# Sample Implementation

from collections import Counter

def random_walk(start_page, link_graph, damping_factor):
    """
    :param start_page: a string with the starting page name
    :param link_graph: a dict mapping page names to "next page" names, e.g. {"page1": ["page2, page5"]}
    :param damping_factor: the probability that the surfer keeps going
    :return: a collections.Counter object with page counts
    """
    counter = Counter()
    page = start_page
    counter[page] += 1
    while keep_surfing(damping_factor):
        page = random_next_page(page, link_graph)
        counter[page] += 1
    return counter

In [ ]:
from collections import Counter

def random_walk(start_page, link_graph, damping_factor):
    """
    :param start_page: a string with the starting page name
    :param link_graph: a dict mapping page names to "next page" names, e.g. {"page1": ["page2, page5"]}
    :param damping_factor: the probability that the surfer keeps going
    :return: a collections.Counter object with page counts
    """
    counter = Counter()

    raise Exception('Implement me!')
    
    return counter

We now have everything we need for our naive implementation of PageRank. Using your **random_walk()** function, repeatedly perform walks on random starting pages and keep track of the overall page counts. You can add one Counter object to another by calling:
>counter.update(another_counter)

In [ ]:
# Sample Implementation

link_graph = {
    'A': ['B'],
    'B': ['C'],
    'C': ['A', 'B']
}

damping_factor = 0.85
total_walks = 10000
overall_counts = Counter()

for i in range(total_walks):
    start_page = np.random.choice(list(link_graph.keys()))
    counts = random_walk(start_page, link_graph, damping_factor)
    overall_counts.update(counts)

print('Raw counts:')
print(overall_counts)
print()
print('PageRanks:')
page_sum = sum(overall_counts.values())
for page in sorted(link_graph):
    print(page, overall_counts[page] / page_sum)

In [ ]:
link_graph = {
    'A': ['B'],
    'B': ['C'],
    'C': ['A', 'B']
}

damping_factor = 0.85
total_walks = 1000
overall_counts = Counter()

###################
# Your code here

raise Exception('Implement me!')

###################

print('Raw counts:')
print(overall_counts)
print()
print('PageRanks:')
page_sum = sum(overall_counts.values())
for page in sorted(link_graph):
    print(page, overall_counts[page] / page_sum)

## A Better Way

While the naive method we just implemented works fine for small graphs, there is a much faster way! In fact, there is a way to compute the long-term stationary distribution of this random walk without using random number generation at all.

### The State Vector and the Adjacency Matrix

The key to our improved implementation will be matrix math.

First, let's represent the current occupancy of each page using a **state vector**, $\mathbf{v}$. We'll number the pages A: 0, B: 1, and C: 2, and begin by placing our surfer on page C.

$$ \mathbf{v} = \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix} $$

Now let's represent our link graph using a device called the **adjacency matrix**, which is a square matrix where each entry $\mathbf{A}_{i,j}$ contains a 1 if an edge is present from node j to i, and otherwise contains a 0. We'll use bold-font $\mathbf{A}$ to distinguish it from the page A.

In this form, each row of the matrix represents the incoming edges for the corresponding node. Each column represents the outgoing edges for the corresponding node.

<img src="graph3.png" width="200"/>

$$ \mathbf{A} = \begin{bmatrix} 0 & 0 & 1 \\ 1 & 0 & 1 \\ 0 & 1 & 0 \end{bmatrix} $$

With the state vector $\mathbf{v}$ and the adjacency matrix $\mathbf{A}$, which mathematical operation (that we have previously learned) will implement a simultaneous "walk" that updates the state vector? Think about this and then expand the hidden cell below to continue.

**$\downarrow$ Expand hidden cell $\downarrow$**

If we multiply the state vector $\mathbf{v}$ by the adjacency matrix $\mathbf{A}$, we get:

$$ \mathbf{v}' = \mathbf{A}\mathbf{v} = \begin{bmatrix} 0 & 0 & 1 \\ 1 & 0 & 1 \\ 0 & 1 & 0 \end{bmatrix} \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix} = \begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix} $$

Our surfer on page $C$ moved to both $A$ and $B$. Now we have two surfers!

Instead of adding new surfers whenever multiple edges are present, let's make the state vector and adjacency matrix stochastic. Instead of tracking a single surfer, imagine tracking an infinite number of surfers and we are representing the fraction of them on each page.

To make the adjacency matrix stochastic, we will need each column to sum to to 1.0.

$$ \mathbf{S} = \begin{bmatrix} 0 & 0 & 0.5 \\ 1 & 0 & 0.5 \\ 0 & 1 & 0 \end{bmatrix} $$

When a page has no outgoing links its column will be all zeros, and in this case we let the surfer start fresh by creating virtual links to all pages. In other words, every entry of the column in this case will contain the value $1/P$. This isn't necessary in our graph above, but we need to remember to implement it in case we encounter a graph that has sink nodes.

Implement code below to create a stochastic adjacency matrix.

In [ ]:
# Sample Implementation

def make_stochastic_adjacency(A):
    S = np.array(A)
    col_sums = np.sum(S, axis=0)
    S[:, col_sums == 0] = 1
    return S / np.sum(S, axis=0)

# Test on a graph that has no outgoing edges for the first page
A = np.array([[0,0,1], [0,0,1], [0,1,0]])

print('Adjacency matrix (notice the first column is all zeros):')
print(A)
print()

print('What we want:')
correct_S = np.array([[1/3, 0, 1/2], [1/3, 0, 1/2], [1/3, 1, 0]])
print(correct_S)
print()

S = make_stochastic_adjacency(A)

print('Your stochastic adjacency matrix:')
print(S)

assert np.allclose(S, correct_S)

In [ ]:
def make_stochastic_adjacency(A):
    """
    :param A: the adjacency matrix, shape (P, P)
    :return: the stochastic adjacency matrix, shape (P, P), all columns sum to 1.0
    """
    raise Exception('Implement me!')

# Test on a graph that has no outgoing edges for the first page
A = np.array([[0,0,1], [0,0,1], [0,1,0]])

print('Adjacency matrix (notice the first column is all zeros):')
print(A)
print()

print('What we want:')
correct_S = np.array([[1/3, 0, 1/2], [1/3, 0, 1/2], [1/3, 1, 0]])
print(correct_S)
print()

S = make_stochastic_adjacency(A)

print('Your stochastic adjacency matrix:')
print(S)

assert np.allclose(S, correct_S)

Now implement one step of our simultaneous, stochastic walk using NumPy:

In [ ]:
# Sample Implementation

def simultaneous_walk(v, S):
    """
    :param v: the state vector, shape (P, 1), where P is the number of pages
    :param S: the stochastic adjacency matrix, shape (P, P)
    :return: the new state vector, shape (P, 1)
    """
    return np.matmul(S, v)

In [ ]:
def simultaneous_walk(v, S):
    """
    :param v: the state vector, shape (P, 1), where P is the number of pages
    :param S: the stochastic adjacency matrix, shape (P, P)
    :return: the new state vector, shape (P, 1)
    """
    raise Exception('Implement me!')

Now implement code that inititalizes $\mathbf{v}$ and $\mathbf{A}$ to match the example above, and perform a single stochastic update step to observe the result:

In [ ]:
# Sample Implementation

A = np.array([
  [0, 0, 1],
  [1, 0, 1],
  [0, 1, 0],  
])
v = np.array([[0], [0], [1]])

S = make_stochastic_adjacency(A)

new_v = simultaneous_walk(v, S)

print(new_v)

In [ ]:
A = # your code here
v = # your code here

S = make_stochastic_adjacency(A)

new_v = simultaneous_walk(v, S)

print(new_v)

We have the code for updating stochastic state when our surfers follow links, but what about the reset operation governed by the damping factor $d$? In this case, the surfer selects a random page instead of following a link. Think about the how to modify our update and then expand the hidden cell below when you want to compare your answer.

**$\downarrow$ Expand hidden cell $\downarrow$**

$$ \mathbf{v}' = (1-d) \begin{bmatrix} 1/3 \\ 1/3 \\ 1/3 \end{bmatrix} + d \begin{bmatrix} 0 & 0 & 0.5 \\ 1 & 0 & 0.5 \\ 0 & 1 & 0 \end{bmatrix} \mathbf{v} $$

This says that with probability $d$ we issue our link-following update, and with the remaining probability $(1-d)$ we reset to a randomly selected page.

Replacing the toy graph numbers above with mathematical symbols we get a generic update formula:

$$ \mathbf{v}' = \frac{1-d}{P} \mathbf{1} + d \mathbf{S} \mathbf{v} $$

where $P$ is the number of pages and $\mathbf{1}$ is a column vector of all ones.

Implement the full update using your previously defined method **simultaneous_walk()**. Make sure you re-normalize the state vector $\mathbf{v}$ after the update so that it sums to 1.0. This is so that floating-point error doesn't cause it drift.

In [ ]:
# Sample Implementation

def full_update(v, S, damping_factor):
    """
    :param v: the state vector, shape (P, 1), where P is the number of pages
    :param S: the stochastic adjacency matrix, shape (P, P)
    :param damping_factor: the probability that the surfer keeps going
    :return: the new state vector, shape (P, 1). it will be normalized to sum to 1
    """
    P = v.shape[0]
    reset_probs = np.ones([P, 1]) / P
    surfing_probs = simultaneous_walk(v, S)
    new_v = (1-damping_factor)*reset_probs + damping_factor*surfing_probs
    return new_v / np.sum(new_v)

In [ ]:
def full_update(v, S, damping_factor):
    """
    :param v: the state vector, shape (P, 1), where P is the number of pages
    :param S: the stochastic adjacency matrix, shape (P, P)
    :param damping_factor: the probability that the surfer keeps going
    :return: the new state vector, shape (P, 1). it will be normalized to sum to 1
    """
    raise Exception('Implement me!')

We almost have enough to implement an accelerated PageRank calculation, except for one crucial question: how do we know when to stop the simulation? Let's stop when the maximum absolute change between any entry in our old state vector and our new state vector is less than some threshold, $eps$.

Implement this using NumPy below:

In [ ]:
# Sample Implementation

def maximum_change(a, b):
    ''' 
    :param a: an ndarray
    :param b: an ndarray
    :return: the maximum value in abs(a - b) 
    '''
    return np.max(np.abs(a - b))

In [ ]:
def maximum_change(a, b):
    ''' 
    :param a: an ndarray
    :param b: an ndarray
    :return: the maximum value in abs(a - b) 
    '''
    # your code here

We're now ready to run our accelerated, stochastic implementation of PageRank. 

Instead of starting with a single surfer on one page, let's instead start with a stochastic state that has equal probability on each page. This is like tracking a huge number of simultaneous surfers:

$$ \mathbf{v} = \begin{bmatrix} 1/3 \\ 1/3 \\ 1/3 \end{bmatrix} $$

As a recap, the pseudo-code is:
* Convert the adjacency matrix, $\mathbf{A}$, to its stochastic version, $\mathbf{S}$
* Initialize a state column vector, $\mathbf{v}$, to all 1/P (where P is the number of pages)
* For step = 1:max_steps
  * save the old state
  * run an update step
  * if maximum absolute change < $eps$, exit loop

In [ ]:
# Sample Implementation

def calc_pagerank(A, damping_factor=0.85, eps=1e-6, max_steps=100):
    """
    :param A: an adjacency matrix (not necessarily stochastic), shape (P, P)
    :param damping_factor: the probability that the surfer keeps going
    :param eps: convergence threshold
    :param max_steps: maximum number of steps to run
    :return: A tuple with two entries: (pagerank vector, step_count)
    """
    P = A.shape[0]
    S = make_stochastic_adjacency(A)
    v = np.ones([P, 1]) / P
    for i in range(max_steps):
        last_v = np.array(v)
        v = full_update(v, S, damping_factor)
        if maximum_change(v, last_v) < eps:
            break
    return v, i

A = np.array([
  [0, 0, 1],
  [1, 0, 1],
  [0, 1, 0],  
])

pageranks, num_steps = calc_pagerank(A, damping_factor=0.85)

print(f'Total steps: {num_steps}')
print()
print('PageRanks:')
print(pageranks)

In [ ]:
def calc_pagerank(A, damping_factor=0.85, eps=1e-6, max_steps=100):
    """
    :param A: an adjacency matrix (not necessarily stochastic), shape (P, P)
    :param damping_factor: the probability that the surfer keeps going
    :param eps: convergence threshold
    :param max_steps: maximum number of steps to run
    :return: A tuple with two entries: (pagerank vector, step_count)
    """
    raise Exception('Implement me!')

A = np.array([
  [0, 0, 1],
  [1, 0, 1],
  [0, 1, 0],  
])

pageranks, num_steps = calc_pagerank(A, damping_factor=0.85)

print(f'Total steps: {num_steps}')
print()
print('PageRanks:')
print(pageranks)

## Larger Graph

Now let's use your PageRank code on the graph we showed in the beginning and compare it to the numbers in that picture. We'll render it again so that you don't need to scroll up. In the picture, the PageRanks are multipled by 100.

In [ ]:
A = np.array([
#    A  B  C  D  E  F g1 g2 g3 g4 g5
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], # A
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0], # B
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # C
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], # D
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], # E
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], # F
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # g1
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # g2
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # g3
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # g4
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # g5
])

pageranks, num_steps = calc_pagerank(A, damping_factor=0.85)

print(f'Total steps: {num_steps}')
print()
print('PageRanks:')
node_names = [chr(o) for o in range(ord('A'), ord('L'))]
dict(zip(node_names, pageranks.ravel()*100))

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/8/8b/PageRanks-Example.jpg/800px-PageRanks-Example.jpg" width="600"/>

## Animation

Run the two cells below to see an animated rendering of your PageRank calculation.

In [ ]:
%%capture

import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

pos = {
    'A': (-1, 1),
    'B': (-0.2,  1),
    'C': (1,  1),
    'D': (-1, 0),
    'E': (0.3, -0.5),
    'F': (1, 0),
    'G1': (-0.8, -0.8),
    'G2': (-0.6, -0.9),
    'G3': (-0.4, -1),
    'G4': (0.6, -1),
    'G5': (0.8, -0.8),
}

G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph())
G = nx.relabel_nodes(G, dict(enumerate(sorted(pos.keys()))))
damping_factor = 0.85

def update(step):
    ax.clear()
    if step == 0:
        pageranks = np.ones(11) / 11
    else:
        pageranks, num_steps = calc_pagerank(A, damping_factor=damping_factor, max_steps=step)
    pageranks = pageranks.ravel()
    nx.draw_networkx(G, pos=pos, with_labels=True,
                     node_color=pageranks, node_size=pageranks*30000,
                     cmap='BuGn', alpha=0.8, edgecolors='k')
    ax.set_title(f'Step: {step}')
    ax.set_xlim(-1.2, 1.3)
    ax.set_ylim(-1.2, 1.5)
    ax.set_xticks([])
    ax.set_yticks([])

fig, ax = plt.subplots(figsize=(10, 6))
anim = animation.FuncAnimation(fig, update, frames=25, interval=500, repeat=True)
HTML(anim.to_jshtml())

In [ ]:
# The output of the cell above is captured to prevent an err plot from rendering inline
# Running this one will show the animation
HTML(anim.to_jshtml())

## Parting Thoughts

What would happen if we tried to use the code we have written for the real web, where we may have over a billion web pages? Our adjacency matrix would use $10^{9} x 10^{9} = 10^{18}$ floats which would not fit in RAM. In a future lesson, we will learn about sparse matrices which will help us solve this problem.

There is also another way to implement this algorithm, which is to realize that the stationary distribution is the principal eigenvector of a modified stochastic adjacency matrix. Knowing this, you could try using NumPy's eigenvector solver instead of coding up the iterative procedure above.

We'll leave the eigenvector approach as an exercise for the reader. Interestingly, the approach we implemented is essentially the "power iteration" approach for finding a principal eigenvector.

Take a moment to ponder the fact that, if you had invented this in 1995, you could have become a billionare instead of Larry Page and Sergey Brin ;-)

Larry and Sergey in the 90s:

![larry_and_sergey](http://archive.fortune.com/assets/i2.cdn.turner.com/money/galleries/2012/news/companies/1203/gallery.greatest-entrepreneurs.fortune/images/larry_page_sergey_brin.jpg)

Larry and Sergey flying their 767 today:

![google_jet](https://www.privatejetcharter.co.uk/wp-content/uploads/2013/11/Boeing-767-200-Google-Jet.jpg)
